# Volumetric maging speed for a given sample size
- sample volume of 1 cm$^3$
- camera Photometric Iris 15, FOV = 21.49 x 12.61 mm
- distance between z-planes 5 or 2 µm 

In [1]:
import numpy as np

In [65]:
MAG = 5 # detection objective magnification
frames_per_s = 2.5 # this includes data writing overhead
FOV_Y_mm, FOV_X_mm = 21.49, 12.61 # Iris 15
FOV_Y_px, FOV_X_px = 5056, 2960 # Iris 15
#FOV_Y_mm, FOV_X_mm = 13.31, 13.31 # Orca Flash 4.3
#FOV_Y_px, FOV_X_px = 2048, 2048 # Orca Flash 4.3
bytes_per_px = 2 # 16-bit camera
FOV_X_sample, FOV_Y_sample = FOV_X_mm/MAG, FOV_Y_mm/MAG
sample_dim_z_mm = 10
sample_dim_x_mm = sample_dim_y_mm = 10
tile_overlap = 0.1 # 10%

In [66]:
z_step = 5e-3 # mm
# N_tiles are allowed to be non-integer, because this is averaged approximation
N_planes = sample_dim_z_mm/z_step
N_tiles_X, N_tiles_Y = sample_dim_x_mm / (FOV_X_sample * (1 - tile_overlap)), sample_dim_y_mm / (FOV_Y_sample * (1 - tile_overlap))
print(f"N_tiles_X, N_tiles_Y = {N_tiles_X:.1f}, {N_tiles_Y:.1f} (can be non-integer, approximation for larger volumes)")

N_tiles_X, N_tiles_Y = 4.4, 2.6 (can be non-integer, approximation for larger volumes)


In [67]:
N_frames_ave = int(N_planes * N_tiles_X * N_tiles_Y) # average over large volumes, N(tiles) can be fractional
N_frames_min = int(N_planes * np.ceil(N_tiles_X) * np.ceil(N_tiles_Y)) # minimum, N(tile) rounded to next integers
total_time_s_ave = int(N_frames_ave / frames_per_s)
total_time_s_min = int(N_frames_min / frames_per_s)
total_GB_ave = N_frames_ave * FOV_Y_px * FOV_X_px * bytes_per_px / 1e9
total_GB_min = N_frames_min * FOV_Y_px * FOV_X_px * bytes_per_px / 1e9
print(f"Imaging time per 1 cm3 sample volume (actual): {total_time_s_min} seconds ({total_time_s_min/60:.1f} minutes)")
print(f"Imaging time per 1 cm3 sample volume (averaged): {total_time_s_ave} seconds ({total_time_s_ave/60:.1f} minutes)\n")

print(f"Dataset size (single channel, actual): {int(total_GB_min)} GB")
print(f"Dataset size (single channel, volume-averaged): {int(total_GB_ave)} GB")

Imaging time per 1 cm3 sample volume (actual): 12000 seconds (200.0 minutes)
Imaging time per 1 cm3 sample volume (averaged): 9111 seconds (151.8 minutes)

Dataset size (single channel, actual): 897 GB
Dataset size (single channel, volume-averaged): 681 GB


Minimum (= actual) time can be larger than (volume) average time because the minimum number of tiles for cannot be less than 1, but on average can be non-integer and < 1. At high magnifications and/or large volumes the difference between the two estimates becomes negligible.